# EDA

In [1]:
import numpy as np
import pandas as pd

import pyspark as ps

In [2]:
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("mov_rec") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
rat_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/ratings.csv', header=True)
        

In [4]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [5]:
rat_rdd.tail(5)

[Row(userId=610, movieId=166534, rating=4.0, timestamp=1493848402),
 Row(userId=610, movieId=168248, rating=5.0, timestamp=1493850091),
 Row(userId=610, movieId=168250, rating=5.0, timestamp=1494273047),
 Row(userId=610, movieId=168252, rating=5.0, timestamp=1493846352),
 Row(userId=610, movieId=170875, rating=3.0, timestamp=1493846415)]

In [6]:
rat_rdd.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
mov_rdd = spark.read.format('com.databricks.spark.csv').\
                            options(header='true',\
                            inferschema='true').\
                            load('data/ml-latest-small/movies.csv', header=True)
        

In [8]:
# tags_rdd = spark.read.format('com.databricks.spark.csv').\
#                              options(header='true',\
#                              inferschema='true').\
#                              load('data/ml-latest-small/tags.csv', header=True)
        

In [9]:
mov_rdd.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [10]:
rat_rdd.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [11]:
mov_rdd.createOrReplaceTempView("mov_rdd")
rat_rdd.createOrReplaceTempView("rat_rdd")

In [12]:
# tags_rdd.show(5)

Leave out timestamp...

In [13]:
mov_surprise = spark.sql(
'''
SELECT ratings.userId, movies.title, ratings.rating 
  FROM mov_rdd AS movies
  JOIN rat_rdd AS ratings
    ON movies.movieId = ratings.movieId
'''
)

In [14]:
mov_surprise.createOrReplaceTempView("mov_surprise")

In [15]:
mov_surprise.show(5)

+------+--------------------+------+
|userId|               title|rating|
+------+--------------------+------+
|     1|    Toy Story (1995)|   4.0|
|     1|Grumpier Old Men ...|   4.0|
|     1|         Heat (1995)|   4.0|
|     1|Seven (a.k.a. Se7...|   5.0|
|     1|Usual Suspects, T...|   5.0|
+------+--------------------+------+
only showing top 5 rows



# Similarity Based Recommender using Surprise

Begin by reimplementing the similarity based recommender.

In [16]:
# # install surprise if not already available
# pip install surprise

In [17]:
from surprise import KNNBasic, KNNWithMeans, SVD

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate

# mov_surprise doesn't contain any zero values
mov_surprise = mov_surprise[mov_surprise['rating']!=0]

# ratings in dataset are from 0.5 to 5
reader = Reader(rating_scale=(0.5, 5))

In [18]:
mov_surprise.head(5)

[Row(userId=1, title='Toy Story (1995)', rating=4.0),
 Row(userId=1, title='Grumpier Old Men (1995)', rating=4.0),
 Row(userId=1, title='Heat (1995)', rating=4.0),
 Row(userId=1, title='Seven (a.k.a. Se7en) (1995)', rating=5.0),
 Row(userId=1, title='Usual Suspects, The (1995)', rating=5.0)]

In [19]:
# convert to pandas DataFrame
# warning, if file is too large, this will likely crash...
mov_pDF = mov_surprise.toPandas()
print(mov_pDF.head(5))

surprise_movie_ratings = Dataset.load_from_df(mov_pDF, reader)

   userId                        title  rating
0       1             Toy Story (1995)     4.0
1       1      Grumpier Old Men (1995)     4.0
2       1                  Heat (1995)     4.0
3       1  Seven (a.k.a. Se7en) (1995)     5.0
4       1   Usual Suspects, The (1995)     5.0


In [20]:
print(mov_pDF.tail(5))

        userId                           title  rating
100831     610                    Split (2017)     4.0
100832     610   John Wick: Chapter Two (2017)     5.0
100833     610                  Get Out (2017)     5.0
100834     610                    Logan (2017)     5.0
100835     610  The Fate of the Furious (2017)     3.0


In [21]:
# treat data as training set
training_movie_data = surprise_movie_ratings.build_full_trainset()

# anything not a user-item rating pair as test
# no data for this, so we can't do an eval on out of sample accuracy
testing_movie_data = training_movie_data.build_anti_testset()

# create instance of recommender algorithm object
# algo = KNNBasic()
algo = SVD()

# fit it on the training data
algo.fit(training_movie_data)

# predict on the test data
predictions = algo.test(testing_movie_data)

In [22]:
# print the first 5 predictions
predictions[:5]

[Prediction(uid=1, iid='Shawshank Redemption, The (1994)', r_ui=3.501556983616962, est=5, details={'was_impossible': False}),
 Prediction(uid=1, iid='Good Will Hunting (1997)', r_ui=3.501556983616962, est=4.688459738687767, details={'was_impossible': False}),
 Prediction(uid=1, iid='Kill Bill: Vol. 1 (2003)', r_ui=3.501556983616962, est=4.404235906425706, details={'was_impossible': False}),
 Prediction(uid=1, iid='Collateral (2004)', r_ui=3.501556983616962, est=4.652124535310926, details={'was_impossible': False}),
 Prediction(uid=1, iid='Talladega Nights: The Ballad of Ricky Bobby (2006)', r_ui=3.501556983616962, est=4.370114940807871, details={'was_impossible': False})]

In [23]:
# print last 5 predictions
predictions[-5:]

[Prediction(uid=610, iid='United States of Leland, The (2003)', r_ui=3.501556983616962, est=3.28440869662284, details={'was_impossible': False}),
 Prediction(uid=610, iid='A Home at the End of the World (2004)', r_ui=3.501556983616962, est=3.5118184235752534, details={'was_impossible': False}),
 Prediction(uid=610, iid='League of Ordinary Gentlemen, A (2004)', r_ui=3.501556983616962, est=3.805430194396398, details={'was_impossible': False}),
 Prediction(uid=610, iid='I Think I Love My Wife (2007)', r_ui=3.501556983616962, est=3.4960909666547404, details={'was_impossible': False}),
 Prediction(uid=610, iid='Man of the Year (1995)', r_ui=3.501556983616962, est=3.668213130455168, details={'was_impossible': False})]

## Cross Validate: SVD

In [24]:
# standard format for cross validation
# algorithm used, ratings dataset, metrics, number of cross validations, verbose controls how much output
cross_validate(SVD(n_factors=5, reg_all=0.005), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8701  0.8752  0.8678  0.8795  0.8599  0.8705  0.0067  
MAE (testset)     0.6705  0.6690  0.6664  0.6749  0.6638  0.6689  0.0038  
Fit time          2.50    2.25    2.50    2.43    2.25    2.39    0.11    
Test time         0.23    0.20    0.19    0.24    0.21    0.21    0.02    


{'test_rmse': array([0.87012303, 0.87520902, 0.86783147, 0.87949249, 0.85986456]),
 'test_mae': array([0.67052924, 0.66901518, 0.66639522, 0.67488141, 0.6637542 ]),
 'fit_time': (2.5017266273498535,
  2.25032377243042,
  2.5048208236694336,
  2.430849313735962,
  2.2534046173095703),
 'test_time': (0.2311854362487793,
  0.20060992240905762,
  0.19200801849365234,
  0.2364346981048584,
  0.21037912368774414)}

## Cross Validtate: KNNBasic

In [25]:
cross_validate(KNNBasic(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9559  0.9522  0.9640  0.9631  0.9646  0.9600  0.0050  
MAE (testset)     0.7287  0.7282  0.7310  0.7329  0.7363  0.7314  0.0030  
Fit time          0.13    0.16    0.17    0.16    0.14    0.15    0.01    
Test time         1.63    1.81    2.02    1.72    1.55    1.75    0.16    


{'test_rmse': array([0.95589241, 0.95221375, 0.96400647, 0.9631295 , 0.9645517 ]),
 'test_mae': array([0.72866785, 0.72824051, 0.73103137, 0.73288176, 0.73628784]),
 'fit_time': (0.13315200805664062,
  0.15950512886047363,
  0.17215657234191895,
  0.16257071495056152,
  0.13786745071411133),
 'test_time': (1.6321558952331543,
  1.8125853538513184,
  2.018242120742798,
  1.7208430767059326,
  1.548384666442871)}

## Cross Validate: KNNWithMeans

In [26]:
cross_validate(KNNWithMeans(k=5), 
               surprise_movie_ratings, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9295  0.9319  0.9270  0.9319  0.9348  0.9310  0.0026  
MAE (testset)     0.7153  0.7148  0.7163  0.7155  0.7193  0.7163  0.0016  
Fit time          0.15    0.18    0.18    0.18    0.18    0.18    0.01    
Test time         1.57    1.58    1.60    1.63    1.88    1.65    0.11    


{'test_rmse': array([0.92945954, 0.93187521, 0.92700969, 0.9319454 , 0.93480844]),
 'test_mae': array([0.71529787, 0.71484955, 0.71629092, 0.71553846, 0.71929988]),
 'fit_time': (0.1480872631072998,
  0.18156862258911133,
  0.18122029304504395,
  0.1832737922668457,
  0.18292021751403809),
 'test_time': (1.573613166809082,
  1.5776593685150146,
  1.6026620864868164,
  1.6321141719818115,
  1.879030704498291)}

# Comparing to Baseline Model

In [34]:
from src import baselines 

# provide basline model our movie ratings
data = surprise_movie_ratings

print("\nGlobal Mean...")
algo = baselines.GlobalMean()
cross_validate(algo, data)

print("\nMeanOfMeans...")
algo = baselines.MeanofMeans()
cross_validate(algo, data)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8721  0.8689  0.8769  0.8609  0.8676  0.8693  0.0053  
MAE (testset)     0.6697  0.6674  0.6752  0.6618  0.6667  0.6682  0.0044  
Fit time          3.00    2.77    2.96    2.80    2.67    2.84    0.12    
Test time         0.35    0.36    0.31    0.30    0.26    0.32    0.04    
{'test_rmse': array([0.87209268, 0.86891766, 0.87692285, 0.86086742, 0.86756916]),
 'test_mae': array([0.66973578, 0.66740167, 0.67518763, 0.66177259, 0.66667806]),
 'fit_time': (3.004497766494751,
  2.7695112228393555,
  2.956921100616455,
  2.801307201385498,
  2.6742496490478516),
 'test_time': (0.34892988204956055,
  0.36215710639953613,
  0.3130302429199219,
  0.3012969493865967,
  0.26192665100097656)}

# Predict results for user & movie

In [35]:
mov_pDF['title'].unique()

array(['Toy Story (1995)', 'Grumpier Old Men (1995)', 'Heat (1995)', ...,
       'Hazard (2005)', 'Blair Witch (2016)', '31 (2016)'], dtype=object)

In [42]:
def movie_title_search(str):
    
    title_lst = list()
    
    for i in mov_pDF['title'].unique():
        
        if str in i:
            title_lst.append(i)
    
    return title_lst  

print(movie_title_search('Matrix'))

['Matrix, The (1999)', 'Matrix Reloaded, The (2003)', 'Matrix Revolutions, The (2003)']
